In [3]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import json
from urlparse import urljoin
import urllib
import urllib2
import re

In [ ]:
url = 'https://www.viamichelin.com/web/Restaurants/Restaurants-United_States?page=1'

r = requests.get(url)
soup = BeautifulSoup(r.text,"html.parser")

In [ ]:
def michelin_restaurant_name (soup):
    name = []
    for row in soup.findAll('div', {'class':'poi-item-name truncate'}):
        try:
            name.append(row.getText().strip('\n'))
            
        except:
            name.append("NA")
    return name

def address (soup):
    address = []
    for row in soup.findAll('div', {'class':'poi-item-address truncate'}):
        try:
            address.append(row.getText().strip('\n').strip('\t').strip('\n'))
            
        except:
            address.append("NA")
    return address

def stars (soup):
    star = []
    for row in soup.findAll(lambda tag: tag.name == 'span' and tag.get('class') == ['poi-item-stars']):
        count = 0
        try:
            for cnt in row.findAll('span', {'class':'star'}):
                count = count+1
            star.append(count)
        except:
            star.append(count)
    return star

In [ ]:
results = pd.DataFrame()

for page in range(1,72):
    url = 'https://www.viamichelin.com/web/Restaurants/Restaurants-United_States?page='+str(page)
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    name_df = pd.DataFrame(michelin_restaurant_name(soup), columns = ['Restaurant_Name'])
    address_df= pd.DataFrame(address(soup), columns = ['Address'])
    star_df = pd.DataFrame(stars(soup), columns = ['Stars'])
    aggregate = pd.concat([name_df, address_df, star_df], axis = 1)
    results = results.append(aggregate)



In [ ]:
results.isnull().sum()

In [ ]:
results.dtypes

In [ ]:
results.shape

In [ ]:
results.to_csv(r'C:\Users\An\Desktop\DSI-DC-2\Portfolio\Michelin\Michelin_Restaurants.csv', sep='\t', encoding='utf-8')

In [ ]:
#Gayot
location = 'NY'
page = 0
gayot_url = 'http://www.gayot.com/restaurants/searchresult.php?search=&rating=13&code=NY&start=50&jump='
r = requests.get(gayot_url)
soup = BeautifulSoup(r.text,"html.parser")


In [ ]:
def gayot_restaurant_name(soup):
    name = []
    for row in soup.findAll('a', {'class':'hoveru'}):
        try:
            name.append(row.getText())
            
        except:
            name.append("NA")
    return name

def gayot_rating(soup):
    rating = []
    
    for row in soup.findAll('td', {'width': '100'}):
        try:
            if row.find('strong').getText() is not None:
                #Orginal format was '13/20'. Pulled out score and convert to float
                rating.append(float(row.find('strong').getText()[0:2]))
            
        except:
            pass
    return rating

def gayot_address(soup):
    address = []
    for row in soup.findAll('td', {'width': '278'}):
        try:
            #Pull address and remove extra tags/characters
            address.append(row.getText()[2:len(row.getText())-11])
            
        except:
            pass
    return address

In [ ]:
gayot_address(soup)

In [ ]:
results = pd.DataFrame()
for city in ['NY']: #Come back and do NY, 140 for CH, 760 for NY
    for rating in range(12,21): #Start with rating 12 as 10 and 11 are too low for Michelin
        for page in range(0,760,10): #The max number of listings per rating per city is 762 (NY with rating 13)
            gayot_url = 'http://www.gayot.com/restaurants/searchresult.php?search=&rating='+str(rating)+'&code='+str(city)+'&start='+str(page)+'&jump='
            r = requests.get(gayot_url)
            soup = BeautifulSoup(r.text,"html.parser")
            name_df = pd.DataFrame(gayot_restaurant_name(soup), columns = ['Restaurant_Name'])
            address_df= pd.DataFrame(gayot_address(soup), columns = ['Address'])
            rating_df = pd.DataFrame(gayot_rating(soup), columns = ['Rating'])
            aggregate = pd.concat([name_df, address_df, rating_df], axis = 1)
            results = results.append(aggregate)

In [ ]:
results.shape

In [ ]:
results.drop_duplicates(inplace=True)

In [ ]:
results.shape

In [ ]:
results.tail()

In [ ]:
results.to_csv(r'C:\Users\An\Desktop\DSI-DC-2\Portfolio\Michelin\NY_Gayot_Restaurants.csv', sep='\t', encoding='utf-8')

In [ ]:
zagat_url = 'https://www.zagat.com/p/washington-dc#filter/addr_city=Washington&score5_food=4.0&vertical=46&orderby=score_food&sort=desc&page=2'

In [ ]:
r = requests.get(zagat_url)
soup = BeautifulSoup(r.text,"html.parser")

def zagat_restaurant_name (soup):
    name = []
    for row in soup.findAll('span', {'class':'content-card__link js-hover-apply'}):
        try:
            name.append(row.getText())
            
        except:
            pass
    return name

def zagat_rating(soup):
    rating = []
    
    for row in soup.findAll('td', {'width': '100'}):
        try:
            if row.find('strong').getText() is not None:
                #Orginal format was '13/20'. Pulled out score and convert to float
                rating.append(float(row.find('strong').getText()[0:2]))
            
        except:
            pass
    return rating

In [ ]:
import re
import sys
import time
import json
import urllib
import sqlite3

sf = pd.DataFrame()
for page in range(1, 50):
    zagat_url= "https://www.zagat.com/proxy/v1.4?vertical=46&orderby=score_food&sort=desc&page="+str(page)+"&city=1021&query=&key=abbc09b7c840c10937a4db331422c98b&mobile_only_content=false&limit=15&m=filter&a=place"
    html = urllib.urlopen(zagat_url).read()
    soup = BeautifulSoup(html,"html.parser")
    city=json.loads(str(soup))
    cityadd = pd.DataFrame(city["data"])
    food = pd.DataFrame()
    decor = pd.DataFrame()
    service = pd.DataFrame()
    for row in range (0,len(city["data"])):
        food = food.append(pd.Series(city['data'][row]['score']['score5_food']),ignore_index=True)
        decor = decor.append(pd.Series(city['data'][row]['score']['score5_decor']),ignore_index=True)
        service = service.append(pd.Series(city['data'][row]['score']['score5_service']),ignore_index=True)
    cityadd['food_rating'] = food
    cityadd['decor_rating'] = decor
    cityadd['service_rating'] = service
    sf = sf.append(cityadd)

sf = sf.reset_index()
del sf['index']

ny = pd.DataFrame()
for page in range(1, 50):
    zagat_url= "https://www.zagat.com/proxy/v1.4?vertical=46&orderby=score_food&sort=desc&page="+str(page)+"&city=1020&query=&key=abbc09b7c840c10937a4db331422c98b&mobile_only_content=false&limit=15&m=filter&a=place"
    html = urllib.urlopen(zagat_url).read()
    soup = BeautifulSoup(html,"html.parser")
    city=json.loads(str(soup))
    cityadd = pd.DataFrame(city["data"])
    food = pd.DataFrame()
    decor = pd.DataFrame()
    service = pd.DataFrame()
    for row in range (0,len(city["data"])):
        food = food.append(pd.Series(city['data'][row]['score']['score5_food']),ignore_index=True)
        decor = decor.append(pd.Series(city['data'][row]['score']['score5_decor']),ignore_index=True)
        service = service.append(pd.Series(city['data'][row]['score']['score5_service']),ignore_index=True)
    cityadd['food_rating'] = food
    cityadd['decor_rating'] = decor
    cityadd['service_rating'] = service
    ny = ny.append(cityadd)

ny = ny.reset_index()
del ny['index']

#Get fewer cities for CH and DC as they are smaller cities. Don't want too much class imbalance
ch = pd.DataFrame()
for page in range(1, 25):
    zagat_url= "https://www.zagat.com/proxy/v1.4?vertical=46&orderby=score_food&sort=desc&page="+str(page)+"&city=1013&query=&key=abbc09b7c840c10937a4db331422c98b&mobile_only_content=false&limit=15&m=filter&a=place"
    html = urllib.urlopen(zagat_url).read()
    soup = BeautifulSoup(html,"html.parser")
    city=json.loads(str(soup))
    cityadd = pd.DataFrame(city["data"])
    food = pd.DataFrame()
    decor = pd.DataFrame()
    service = pd.DataFrame()
    for row in range (0,len(city["data"])):
        food = food.append(pd.Series(city['data'][row]['score']['score5_food']),ignore_index=True)
        decor = decor.append(pd.Series(city['data'][row]['score']['score5_decor']),ignore_index=True)
        service = service.append(pd.Series(city['data'][row]['score']['score5_service']),ignore_index=True)
    cityadd['food_rating'] = food
    cityadd['decor_rating'] = decor
    cityadd['service_rating'] = service
    ch = ch.append(cityadd)

ch = ch.reset_index()
del ch['index']

dc = pd.DataFrame()
for page in range(1, 25):
    #zagat_url= "https://www.zagat.com/proxy/v1.4?addr_city=Washington&score5_food=4.0&vertical=46&orderby=score_food&sort=desc&page="+str(page)+"&city=1024&query=&key=abbc09b7c840c10937a4db331422c98b&mobile_only_content=false&limit=15&m=filter&a=place"
    zagat_url= "https://www.zagat.com/proxy/v1.4?vertical=46&orderby=score_food&sort=desc&page="+str(page)+"&city=1024&query=&key=abbc09b7c840c10937a4db331422c98b&mobile_only_content=false&limit=15&m=filter&a=place"
    html = urllib.urlopen(zagat_url).read()
    soup = BeautifulSoup(html,"html.parser")
    city=json.loads(str(soup))
    cityadd = pd.DataFrame(city["data"])
    food = pd.DataFrame()
    decor = pd.DataFrame()
    service = pd.DataFrame()
    for row in range (0,len(city["data"])):
        food = food.append(pd.Series(city['data'][row]['score']['score5_food']),ignore_index=True)
        decor = decor.append(pd.Series(city['data'][row]['score']['score5_decor']),ignore_index=True)
        service = service.append(pd.Series(city['data'][row]['score']['score5_service']),ignore_index=True)
    cityadd['food_rating'] = food
    cityadd['decor_rating'] = decor
    cityadd['service_rating'] = service
    dc = dc.append(cityadd)

dc = dc.reset_index()
del dc['index']

In [ ]:
columns = ['title', 'cuisine', 'cost', 'price_level', 'food_rating', 'decor_rating', 'service_rating']
sf = sf[columns]
ny = ny[columns]
ch = ch[columns]
dc = dc[columns]

In [ ]:
ny.to_csv(r'C:\Users\An\Desktop\DSI-DC-2\Portfolio\Michelin\NY_Zagat_Restaurants.csv', sep='\t', encoding='utf-8')
sf.to_csv(r'C:\Users\An\Desktop\DSI-DC-2\Portfolio\Michelin\SF_Zagat_Restaurants.csv', sep='\t', encoding='utf-8')
ch.to_csv(r'C:\Users\An\Desktop\DSI-DC-2\Portfolio\Michelin\CH_Zagat_Restaurants.csv', sep='\t', encoding='utf-8')
dc.to_csv(r'C:\Users\An\Desktop\DSI-DC-2\Portfolio\Michelin\DC_Zagat_Restaurants.csv', sep='\t', encoding='utf-8')

## Read, standardize and join csv data

In [56]:
ny_gayot = pd.read_csv('NY_Gayot_Restaurants.csv',sep='\t', encoding='utf-8')
sf_gayot = pd.read_csv('SF_Gayot_Restaurants.csv',sep='\t', encoding='utf-8')
ch_gayot = pd.read_csv('CH_Gayot_Restaurants.csv',sep='\t', encoding='utf-8')
dc_gayot = pd.read_csv('DC_Gayot_Restaurants.csv',sep='\t', encoding='utf-8')
ny_zagat = pd.read_csv('NY_Zagat_Restaurants.csv',sep='\t', encoding='utf-8')
sf_zagat = pd.read_csv('SF_Zagat_Restaurants.csv',sep='\t', encoding='utf-8')
ch_zagat = pd.read_csv('CH_Zagat_Restaurants.csv',sep='\t', encoding='utf-8')
dc_zagat = pd.read_csv('DC_Zagat_Restaurants.csv',sep='\t', encoding='utf-8')

michelin = pd.read_csv('Michelin_Restaurants.csv',sep='\t', encoding='utf-8')
award = pd.read_csv('James_Beard_Awards.csv')

ny_zagat.rename(columns = {'title':'Restaurant_Name'}, inplace = True)
ch_zagat.rename(columns = {'title':'Restaurant_Name'}, inplace = True)
sf_zagat.rename(columns = {'title':'Restaurant_Name'}, inplace = True)
dc_zagat.rename(columns = {'title':'Restaurant_Name'}, inplace = True)
ny_gayot['Restaurant_Name'] = ny_gayot['Restaurant_Name'].map(lambda x: x.lower().lstrip().rstrip())
ch_gayot['Restaurant_Name'] = ch_gayot['Restaurant_Name'].map(lambda x: x.lower().lstrip().rstrip())
sf_gayot['Restaurant_Name'] = sf_gayot['Restaurant_Name'].map(lambda x: x.lower().lstrip().rstrip())
dc_gayot['Restaurant_Name'] = dc_gayot['Restaurant_Name'].map(lambda x: x.lower().lstrip().rstrip())
ny_zagat['Restaurant_Name'] = ny_zagat['Restaurant_Name'].map(lambda x: x.lower().lstrip().rstrip())
ch_zagat['Restaurant_Name'] = ch_zagat['Restaurant_Name'].map(lambda x: x.lower().lstrip().rstrip())
dc_zagat['Restaurant_Name'] = dc_zagat['Restaurant_Name'].map(lambda x: x.lower().lstrip().rstrip())
sf_zagat['Restaurant_Name'] = sf_zagat['Restaurant_Name'].map(lambda x: x.lower().lstrip().rstrip())
michelin['Restaurant_Name'] = michelin['Restaurant_Name'].map(lambda x: x.lower().lstrip().rstrip())
award['Restaurant_Name'] = award['Restaurant_Name'].map(lambda x: x.lower().lstrip().rstrip())

#Drop chain restaurants
ny_gayot.drop_duplicates(['Restaurant_Name'],keep=False, inplace = True)
ch_gayot.drop_duplicates(['Restaurant_Name'],keep=False, inplace = True)
sf_gayot.drop_duplicates(['Restaurant_Name'],keep=False, inplace = True)
dc_gayot.drop_duplicates(['Restaurant_Name'],keep=False, inplace = True)
ny_zagat.drop_duplicates(['Restaurant_Name'],keep=False, inplace = True)
ch_zagat.drop_duplicates(['Restaurant_Name'],keep=False, inplace = True)
sf_zagat.drop_duplicates(['Restaurant_Name'],keep=False, inplace = True)
dc_zagat.drop_duplicates(['Restaurant_Name'],keep=False, inplace = True)

ny_gayot.reset_index(inplace=True, drop = True)
ch_gayot.reset_index(inplace=True, drop = True)
sf_gayot.reset_index(inplace=True, drop = True)
dc_gayot.reset_index(inplace=True, drop = True)
ny_zagat.reset_index(inplace=True, drop = True)
ch_zagat.reset_index(inplace=True, drop = True)
sf_zagat.reset_index(inplace=True, drop = True)
dc_zagat.reset_index(inplace=True, drop = True)
michelin.reset_index(inplace=True, drop = True)

del ny_gayot['Unnamed: 0']
del sf_gayot['Unnamed: 0']
del ch_gayot['Unnamed: 0']
del dc_gayot['Unnamed: 0']
del ny_zagat['Unnamed: 0']
del sf_zagat['Unnamed: 0']
del ch_zagat['Unnamed: 0']
del dc_zagat['Unnamed: 0']
del michelin['Unnamed: 0']

In [57]:
def zip(x):
    return x[len(x)-5:len(x)]

michelin['zip']= michelin['Address'].map(zip)

In [58]:
def ny (x):
    if (x[0]=='1'):
        return 1
    else:
        return 0
    
def ch (x):
    if (x[0]=='6') or (x=='icago'):
        return 1
    else:
        return 0
    
def sf (x):
    if (x[0] == '9'):
        return 1
    else:
        return 0

#Indicates City of the row
michelin['ny'] = michelin['zip'].apply(ny)
michelin['ch'] = michelin['zip'].apply(ch)
michelin['sf'] = michelin['zip'].apply(sf)

In [59]:
ny = pd.merge(michelin[michelin['ny']==1], ny_gayot, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')
ny = pd.merge(ny, ny_zagat, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')
sf = pd.merge(michelin[michelin['sf']==1], sf_gayot, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')
sf = pd.merge(sf, sf_zagat, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')
ch = pd.merge(michelin[michelin['ch']==1], ch_gayot, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')
ch = pd.merge(ch, ch_zagat, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')
dc = pd.merge(dc_zagat, dc_gayot, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')

ny = pd.merge(ny, award, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')
sf = pd.merge(sf, award, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')
ch = pd.merge(ch, award, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')
dc = pd.merge(dc, award, left_on = 'Restaurant_Name', right_on = 'Restaurant_Name', how = 'left')

ny['Award_Past_Three_Years'].fillna(value = 0, inplace = True)
sf['Award_Past_Three_Years'].fillna(value = 0, inplace = True)
ch['Award_Past_Three_Years'].fillna(value = 0, inplace = True)
dc['Award_Past_Three_Years'].fillna(value = 0, inplace = True)

In [60]:
ny['Stars'].sum()/len(ny)

0.11439114391143912

In [61]:
sf['Stars'].sum()/len(sf)

0.13598326359832635

In [62]:
ch['Stars'].sum()/len(ch)

0.09491525423728814

In [63]:
#Drop major non-DC restaurants
dc = dc[dc['Restaurant_Name']!='the inn at little washington']
dc = dc[dc['Restaurant_Name']!='restaurant eve']
dc = dc[dc['Restaurant_Name']!='chima brazilian steakhouse']
dc = dc[dc['Restaurant_Name']!='tachibana japanese restaurant']

dc.drop_duplicates(['Restaurant_Name'], keep=False, inplace = True)
dc.reset_index(inplace=True, drop = True)

In [64]:
train = pd.concat([ny,sf], ignore_index = True)[['Restaurant_Name', 'Stars', 'Rating', 'cost', 'food_rating', 'decor_rating', 'service_rating', 'Award_Past_Three_Years']]
#price_level_dummies = pd.get_dummies(train.price_level)
#train = pd.concat([train[['Restaurant_Name', 'Stars', 'Rating', 'cost', 'food_rating', 'decor_rating', 'service_rating']], price_level_dummies], axis = 1)

## Need to impute Rating from food_rating

In [65]:
train.groupby('food_rating')['Rating'].median()

food_rating
4.3    14.0
4.4    14.0
4.5    14.0
4.6    14.0
4.7    14.0
4.8    16.0
4.9    18.0
Name: Rating, dtype: float64

In [66]:
#Searching for nulls wasn't working so I had fill with a number
import numpy as np
train['Rating'].fillna(-5.0, inplace=True)

# for i in range(0,len(train)):
#     if (train.loc[i,('Rating')] == -5.0) and (train.loc[i,('food_rating')]): 
#         if train.loc[i,('food_rating')] < 4.8:
#             train.loc[i,('Rating')] = 14.0 #Assume 13.5
#         elif train.loc[i,('food_rating')] == 4.8:
#             train.loc[i,('Rating')] = 16.0
#         else:
#             train.loc[i,('Rating')] = 18.0

for i in range(0,len(train)):
    if (train.loc[i,('Rating')] == -5.0) and (train.loc[i,('food_rating')]): 
        if train.loc[i,('food_rating')] < 4.7:
            train.loc[i,('Rating')] = 14.0 #Assume 13.5
        elif train.loc[i,('food_rating')] == 4.7:
            train.loc[i,('Rating')] = 14.5
        elif train.loc[i,('food_rating')] == 4.8:
            train.loc[i,('Rating')] = 16.5
        else:
            train.loc[i,('Rating')] = 17.0

In [67]:
ch.groupby('food_rating')['Rating'].mean()

food_rating
4.4          NaN
4.5    13.833333
4.6    14.307692
4.7    14.818182
4.8    15.500000
4.9          NaN
Name: Rating, dtype: float64

In [68]:
ch['Rating'].fillna(-5.0, inplace=True)

for i in range(0,len(ch)):
    if (ch.loc[i,('Rating')] == -5.0) and (ch.loc[i,('food_rating')]): 
        if ch.loc[i,('food_rating')] < 4.6:
            ch.loc[i,('Rating')] = 13.5
        elif ch.loc[i,('food_rating')] <4.8:
            ch.loc[i,('Rating')] = 14.5
        else:
            ch.loc[i,('Rating')] = 15.5

In [69]:
#Used mean as median doesn't give much insight. Median is 14.0 for all food_ratings
dc.groupby('food_rating')['Rating'].mean() 

food_rating
4.3    13.666667
4.4    13.600000
4.5    14.133333
4.6    14.111111
4.7    14.800000
4.8    14.250000
Name: Rating, dtype: float64

In [70]:
dc['Rating'].fillna(-5.0, inplace=True)

for i in range(0,len(dc)):
    if (dc.loc[i,('Rating')] == -5.0) and (dc.loc[i,('food_rating')]): 
        if dc.loc[i,('food_rating')] < 4.7:
            dc.loc[i,('Rating')] = 14.0
#        elif dc.loc[i,('food_rating')] < 4.7:
#            dc.loc[i,('Rating')] = 13.0 #Assume that 4.4 should be 13.0 as well
        else:
            dc.loc[i,('Rating')] = 15.0

In [71]:
#Create dataframe to use for DC predictions
#dc_price_level_dummies = pd.get_dummies(dc.price_level)
#dc_test = pd.concat([dc[['Rating', 'cost', 'food_rating', 'decor_rating', 'service_rating']], dc_price_level_dummies], axis = 1)
dc_test = dc[['Rating', 'cost', 'food_rating', 'decor_rating', 'service_rating', 'Award_Past_Three_Years']]
dc_test.dropna(inplace=True)
dc_test.reset_index(inplace =True, drop = True)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [72]:
#Keep columns as 
#ch_price_level_dummies = pd.get_dummies(ch.price_level)
#test = pd.concat([ch[['Restaurant_Name', 'Stars', 'Rating', 'cost', 'food_rating', 'decor_rating', 'service_rating']], price_level_dummies], axis = 1)
train.dropna(inplace = True)
#X_train = train[['Rating', 'cost', 'food_rating', 'decor_rating', 'service_rating']]
X_train = train[train.columns -['Restaurant_Name'] - ['Stars']]
y_train = train['Stars']

#ch_price_level_dummies = pd.get_dummies(ch.price_level)
#test = pd.concat([ch[['Stars', 'Rating', 'cost', 'food_rating', 'decor_rating', 'service_rating']], ch_price_level_dummies], axis = 1)
test = ch[['Stars', 'Rating', 'cost', 'food_rating', 'decor_rating', 'service_rating', 'Award_Past_Three_Years']]
test.dropna(inplace=True)
#train[train.columns -['Restaurant_Name'] - ['Stars']]
X_test = test[test.columns - ['Stars']]
#x_test = test[['Rating', 'cost', 'food_rating','decor_rating', 'service_rating','M', 'VE']]
y_test = test['Stars']

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()


In [73]:
#Takes model.predict(dc_test) as input
def dc_predictions (x):
    dc_result = pd.DataFrame(x)
    dc_result.rename(columns = {0:'predicted_stars'}, inplace=True)
    dc_dummy = dc_test.reset_index(drop = True)
    temp = pd.concat([dc_result, dc_dummy], axis = 1)
    dc_final = pd.merge(dc, temp, on =['Rating','cost','food_rating','decor_rating','service_rating'] )
    dc_final = dc_final[['Restaurant_Name', 'predicted_stars']]
    print '\n',dc_final[dc_final['predicted_stars'] >0]

In [81]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

rf = RandomForestClassifier(class_weight = 'balanced')
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
dc_predictions(rf.predict(dc_test))

Accuracy 0.820512820513
Confusion Matrix 
[[60  4  1  0]
 [ 5  4  1  0]
 [ 0  2  0  0]
 [ 0  0  1  0]]
Classification Report 
             precision    recall  f1-score   support

        0.0       0.92      0.92      0.92        65
        1.0       0.40      0.40      0.40        10
        2.0       0.00      0.00      0.00         2
        3.0       0.00      0.00      0.00         1

avg / total       0.82      0.82      0.82        78


Empty DataFrame
Columns: [Restaurant_Name, predicted_stars]
Index: []


In [ ]:
from sklearn.grid_search import GridSearchCV

param_grid = {'n_estimators': [5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20], 'criterion':['gini', 'entropy'], 'max_depth':[3,5,7,10,12,15], 'min_samples_split': [2,3,4], 'min_samples_leaf':[1,2,3] }
gsrf = GridSearchCV(RandomForestClassifier(), param_grid, verbose = 2, cv= 6, n_jobs = -1)
gsrf.fit(X_train, y_train)
y_pred = gsrf.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
gsrf.best_params_

In [ ]:
rf_best = RandomForestClassifier(criterion= 'gini', max_depth= 3, min_samples_leaf= 2, min_samples_split = 3, n_estimators = 8)
rf_best.fit(X_train, y_train)
y_pred = rf_best.predict(X_test)
dc_predictions(rf_best.predict(dc_test))

In [ ]:
rf_best.feature_importances_

In [ ]:
train[train.columns -['Restaurant_Name'] - ['Stars']].head(1)

In [76]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

ex = ExtraTreesClassifier()
ex.fit(X_train, y_train)
y_pred = ex.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
dc_predictions(ex.predict(dc_test))

Accuracy 0.884615384615
Confusion Matrix 
[[62  2  1  0]
 [ 4  5  1  0]
 [ 0  1  1  0]
 [ 0  0  0  1]]
Classification Report 
             precision    recall  f1-score   support

        0.0       0.94      0.95      0.95        65
        1.0       0.62      0.50      0.56        10
        2.0       0.33      0.50      0.40         2
        3.0       1.00      1.00      1.00         1

avg / total       0.88      0.88      0.88        78


Empty DataFrame
Columns: [Restaurant_Name, predicted_stars]
Index: []


In [51]:
param_grid = {'n_estimators': [5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20], 'criterion':['gini', 'entropy'], 'max_depth':[None,3,5,7,10,12,15], 'min_samples_split': [2,3,4,5,6], 'min_samples_leaf':[1,2,3] }
gsex = GridSearchCV(ExtraTreesClassifier(), param_grid, verbose = 2, n_jobs = -1)
gsex.fit(X_train, y_train)
y_pred = gsex.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
gsex.best_params_

NameError: name 'GridSearchCV' is not defined

In [ ]:
ex_best = ExtraTreesClassifier(criterion='entropy', max_depth = 12, min_samples_leaf = 1, min_samples_split = 6, n_estimators = 9)
ex_best.fit(X_train, y_train)
y_pred = ex_best.predict(X_test)
dc_predictions(ex_best.predict(dc_test))

In [77]:
from sklearn import svm

svm_rbf = svm.SVC()
svm_rbf.fit(X_train, y_train)
y_pred = svm_rbf.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
dc_predictions(svm_rbf.predict(dc_test))

Accuracy 0.846153846154
Confusion Matrix 
[[64  1  0  0]
 [ 7  2  0  1]
 [ 0  2  0  0]
 [ 0  0  1  0]]
Classification Report 
             precision    recall  f1-score   support

        0.0       0.90      0.98      0.94        65
        1.0       0.40      0.20      0.27        10
        2.0       0.00      0.00      0.00         2
        3.0       0.00      0.00      0.00         1

avg / total       0.80      0.85      0.82        78


Empty DataFrame
Columns: [Restaurant_Name, predicted_stars]
Index: []


In [ ]:
svm_l = svm.SVC(kernel = 'linear')
svm_l.fit(X_train, y_train)
y_pred = svm_l.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
dc_predictions(svm_l.predict(dc_test))

In [ ]:
svm_p = svm.SVC(kernel = 'poly')
svm_p.fit(X_train, y_train)
y_pred = svm_p.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
dc_predictions(svm_p.predict(dc_test))

In [78]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
dc_predictions(dt.predict(dc_test))

Accuracy 0.846153846154
Confusion Matrix 
[[60  4  1  0]
 [ 6  4  0  0]
 [ 0  1  1  0]
 [ 0  0  0  1]]
Classification Report 
             precision    recall  f1-score   support

        0.0       0.91      0.92      0.92        65
        1.0       0.44      0.40      0.42        10
        2.0       0.50      0.50      0.50         2
        3.0       1.00      1.00      1.00         1

avg / total       0.84      0.85      0.84        78


Empty DataFrame
Columns: [Restaurant_Name, predicted_stars]
Index: []


In [79]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bdt = BaggingClassifier(DecisionTreeClassifier())
bdt.fit(X_train, y_train)
y_pred = bdt.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
dc_predictions(bdt.predict(dc_test))

Accuracy 0.858974358974
Confusion Matrix 
[[61  3  1  0]
 [ 5  5  0  0]
 [ 0  1  1  0]
 [ 0  0  1  0]]
Classification Report 
             precision    recall  f1-score   support

        0.0       0.92      0.94      0.93        65
        1.0       0.56      0.50      0.53        10
        2.0       0.33      0.50      0.40         2
        3.0       0.00      0.00      0.00         1

avg / total       0.85      0.86      0.85        78


Empty DataFrame
Columns: [Restaurant_Name, predicted_stars]
Index: []


In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
dc_predictions(lr.predict(dc_test))

In [55]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
kn = KNeighborsClassifier()
kn.fit(X_train, y_train)
y_pred = kn.predict(X_test)
print 'Accuracy', accuracy_score(y_test, y_pred)
print 'Confusion Matrix','\n',  confusion_matrix(y_test, y_pred)
print 'Classification Report','\n', classification_report(y_test, y_pred)
dc_predictions(kn.predict(dc_test))

Accuracy 0.0128205128205
Confusion Matrix 
[[ 0  0  0 65]
 [ 0  0  0 10]
 [ 0  0  0  2]
 [ 0  0  0  1]]
Classification Report 
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        65
        1.0       0.00      0.00      0.00        10
        2.0       0.00      0.00      0.00         2
        3.0       0.01      1.00      0.03         1

avg / total       0.00      0.01      0.00        78


                                    Restaurant_Name  predicted_stars
0                      monocacy crossing restaurant              3.0
1                                   rasika west end              3.0
2                                   rasika west end              3.0
3                                            rasika              3.0
4                                            rasika              3.0
5                                             fiola              3.0
6                                          corduroy              3.0